## Process MFASS data

Sequences of the reference and alternative are already provided for this dataset. Write function to extract different components.

This is an exon skipping setting.

In [1]:
import sys
sys.path.append("../")

from kipoi.data import DataLoader
from kipoi.data_utils import numpy_collate
from concise.preprocessing import encodeDNA
from keras.models import load_model
from MFASS_dataloader import MFASS_Exon_Dataset
from scipy.stats import spearmanr, pearsonr

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.metrics import precision_recall_curve

import mmsplice
from mmsplice.generic import onehot, logit, expit
from mmsplice import MMSplice
from mmsplice.utils.postproc import transform

%load_ext autoreload
%autoreload 2

projectDIR = "../../data/"

Using TensorFlow backend.


In [2]:
dtfile = projectDIR + "mfass/snv_data_clean.txt"

In [3]:
mfass = pd.read_table(dtfile)

In [4]:
mfass.head()

,id,ensembl_id,sub_id,chr,start,end,strand,intron1_len,exon_len,intron2_len,...,v2_dpsi_R1,v2_dpsi_R2,nat_v1_index,nat_v2_index,nat_seq,v1_dpsi,v2_dpsi,delta_dpsi,strong_lof,strong_lof_v1
0,ENSE00000332835_000,ENSE00000332835,000,chrX,48837777,48837946,-,51,57,62,...,0.000000,0.000000,0.848391,0.830145,GAGGGAGCATGCTTCAGGCAGCCCTTCCGGCATCTACCAGCCATGC...,0.0,0.000000,0.0,False,False
1,ENSE00000332835_007,ENSE00000332835,007,chrX,48837777,48837946,-,51,57,62,...,-0.053480,0.141578,0.848391,0.830145,GAGGGAGCATGCTTCAGGCAGCCCTTCCGGCATCTACCAGCCATGC...,NaN,0.044049,NaN,False,NaN
2,ENSE00000338771_000,ENSE00000338771,000,chr15,41029796,41029965,-,30,99,41,...,0.000000,0.000000,NaN,0.712929,TTTTAAAACCTCTTATACACACCTTGATCTTTTTGTGGCAGGTCTC...,NaN,0.000000,NaN,False,NaN
3,ENSE00000338771_002,ENSE00000338771,002,chr15,41029796,41029965,-,30,99,41,...,-0.599316,-0.806640,NaN,0.712929,TTTTAAAACCTCTTATACACACCTTGATCTTTTTGTGGCAGGTCTC...,NaN,-0.702978,NaN,True,NaN
4,ENSE00000338771_003,ENSE00000338771,003,chr15,41029796,41029965,-,30,99,41,...,0.268491,0.187646,NaN,0.712929,TTTTAAAACCTCTTATACACACCTTGATCTTTTTGTGGCAGGTCTC...,NaN,0.228068,NaN,False,NaN


In [5]:
mfass.shape

(32669, 54)

In [6]:
dl = MFASS_Exon_Dataset(dtfile)

In [7]:
dl = DataLoader(dl, batch_size=dl.__len__(), collate_fn=numpy_collate, shuffle=False)
dl = iter(dl)

In [8]:
dt = next(dl)

In [9]:
dt['inputs']["seq"]

array(['GAGGGAGCATGCTTCAGGCAGCCCTTCCGGCATCTACCAGCCATGCCTGCTTTCTCCCACAGGAGGTGAAGGACACAGTGGATGGGCAGAGGATCCTGGAGAAGAAGGGCAGTGCTGCGGTAAGACAGAGCGGTGCCCAAGCACGGCTCCCTCCTACCTGTAGCCTCCCT',
       'GAGGGAGCATGCTTCAGGCAGCCCTTCCGGCATCTACCAGCCATGCCTGCTTTCTCCCACAGGAGGTGAAGGACACAGTGGATGGGCAGAGGATCCTGGAGAAGAAGGGCAGTGCTGCGGTAAGACAGAGCGGTGCCCAAGCACGGCTCCCTCCTACCTGTAGCCTCCCT',
       'TTTTAAAACCTCTTATACACACCTTGATCTTTTTGTGGCAGGTCTCTCACCTGAGCTGGCTAGAAAAAAAAACTGCTACAGCCTTGCTTGAAAGCCCTCTCAGTGCCACTGTGGAAGATGCCCTCCAGAGCTTCCTAAAGGTATGAGAGGGGCAGGTGGCAGGCAAAAAC',
       ...,
       'CTTTGCTCACTTTCACTCACTCTCTTTTTCATTTCTCCCCCCAGTTTCACCGCAAATGTGAATTATCAACGTTGTGTGACGGTGGGGAACTCAGAGACCACATCTTACTGCCCACCTCCATATGCCCCATCACCCGGGTAAGTGCTCCTGCCCCGGCGAGGGCTGTAGCT',
       'CTTTGCTCACTTTCACTCACTCTCTTTTTCATTTCTCCCCCCAGTTTCACCGCAAATGTGAATTATCAACGTTGTGTGACGGTGGGGAACTCAGAGACCACATCTTACTGCCCACCTCCATATGCCCCATCACCCGGGTAAGTGCTCCTGCCCCGGCGAGGGCTGTAGCT',
       'CTTTGCTCACTTTCACTCACTCTCTTTTTCATTTCTCCCCCCAGTTTCACCGCAAATGTGAATTATCAACGTTGT

In [10]:
dt['inputs']["seq"]

array(['GAGGGAGCATGCTTCAGGCAGCCCTTCCGGCATCTACCAGCCATGCCTGCTTTCTCCCACAGGAGGTGAAGGACACAGTGGATGGGCAGAGGATCCTGGAGAAGAAGGGCAGTGCTGCGGTAAGACAGAGCGGTGCCCAAGCACGGCTCCCTCCTACCTGTAGCCTCCCT',
       'GAGGGAGCATGCTTCAGGCAGCCCTTCCGGCATCTACCAGCCATGCCTGCTTTCTCCCACAGGAGGTGAAGGACACAGTGGATGGGCAGAGGATCCTGGAGAAGAAGGGCAGTGCTGCGGTAAGACAGAGCGGTGCCCAAGCACGGCTCCCTCCTACCTGTAGCCTCCCT',
       'TTTTAAAACCTCTTATACACACCTTGATCTTTTTGTGGCAGGTCTCTCACCTGAGCTGGCTAGAAAAAAAAACTGCTACAGCCTTGCTTGAAAGCCCTCTCAGTGCCACTGTGGAAGATGCCCTCCAGAGCTTCCTAAAGGTATGAGAGGGGCAGGTGGCAGGCAAAAAC',
       ...,
       'CTTTGCTCACTTTCACTCACTCTCTTTTTCATTTCTCCCCCCAGTTTCACCGCAAATGTGAATTATCAACGTTGTGTGACGGTGGGGAACTCAGAGACCACATCTTACTGCCCACCTCCATATGCCCCATCACCCGGGTAAGTGCTCCTGCCCCGGCGAGGGCTGTAGCT',
       'CTTTGCTCACTTTCACTCACTCTCTTTTTCATTTCTCCCCCCAGTTTCACCGCAAATGTGAATTATCAACGTTGTGTGACGGTGGGGAACTCAGAGACCACATCTTACTGCCCACCTCCATATGCCCCATCACCCGGGTAAGTGCTCCTGCCCCGGCGAGGGCTGTAGCT',
       'CTTTGCTCACTTTCACTCACTCTCTTTTTCATTTCTCCCCCCAGTTTCACCGCAAATGTGAATTATCAACGTTGT

In [11]:
dt['inputs']['intronr_len']

array([51, 51, 30, ..., 33, 33, 33])

## Apply prediction model

In [12]:
model = MMSplice(
    exon_cut_l=0,
    exon_cut_r=0,
    acceptor_intron_cut=6,
    donor_intron_cut=6,
    acceptor_intron_len=50,
    acceptor_exon_len=3,
    donor_exon_len=5,
    donor_intron_len=13)

/opt/modules/i12g/anaconda/3-5.0.1/envs/splicing/lib/python3.6/site-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [13]:
ref = model.predict_on_unsplitted_batch(dt['inputs'])

/opt/modules/i12g/anaconda/3-5.0.1/envs/splicing/lib/python3.6/site-packages/mmsplice/mmsplice.py:157: UserWarning: None GT donor
  warnings.warn("None GT donor", UserWarning)


In [14]:
alt = model.predict_on_unsplitted_batch(dt['inputs_mut'])

/opt/modules/i12g/anaconda/3-5.0.1/envs/splicing/lib/python3.6/site-packages/mmsplice/mmsplice.py:157: UserWarning: None GT donor
  warnings.warn("None GT donor", UserWarning)
/opt/modules/i12g/anaconda/3-5.0.1/envs/splicing/lib/python3.6/site-packages/mmsplice/mmsplice.py:159: UserWarning: None AG donor
  warnings.warn("None AG donor", UserWarning)


In [15]:
X = alt - ref

In [16]:
exon_overlap = np.logical_or(np.logical_and(X[:,1]!=0, X[:,2]!=0), np.logical_and(X[:,2]!=0, X[:,3]!=0))

acceptor_intron_overlap = np.logical_and(X[:,0]!=0, X[:,1]!=0)
donor_intron_overlap = np.logical_and(X[:,3]!=0, X[:,4]!=0)

X = np.hstack([X, (X[:,2]*exon_overlap).reshape(-1,1)])
X = np.hstack([X, (X[:,4]*donor_intron_overlap).reshape(-1,1)])
X = np.hstack([X, (X[:,0]*acceptor_intron_overlap).reshape(-1,1)])

In [17]:
from sklearn.externals import joblib
huber = joblib.load(projectDIR + 'vexseq/scale_model.pkl')

pred_delta = huber.predict(X)

In [18]:
keep = (mfass.nat_v2_index >=0.5) & (mfass.category=='mutant') & (~np.isnan(mfass.v2_dpsi))

In [19]:
pearsonr(pred_delta[keep], mfass.v2_dpsi[keep])

(0.42129621882803897, 0.0)

In [20]:
ref_psi = mfass.nat_v2_index
pred_delta += logit(ref_psi)
pred_delta = expit(pred_delta)
pred_delta -= ref_psi

In [21]:
pred = pd.concat([mfass.strong_lof[keep], 
                 pd.Series(pred_delta)[keep], 
                 mfass.v2_dpsi[keep], 
                mfass.label[keep],
                 mfass.id[keep]], axis=1, ignore_index=True)

In [22]:
pred.to_csv(projectDIR+"/mfass/MMSplice_pred.txt", index=False)